In [1]:
# ---------------- Student Progress Reports (PDFs) ----------------
import os
import re
import numpy as np
import openpyxl as xl
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import pandas as pd

# ---------------- Excel Processing Section ----------------
file_path = "S2PythonInternal.xlsx"   # Input Excel file containing student marks
wb = xl.load_workbook(file_path)
sheet = wb.active

# Directory to save individual student PDF reports
report_dir = "Student_Reports"
os.makedirs(report_dir, exist_ok=True)

# ---------------- Load Data with Pandas ----------------
file_path = "S2PythonInternal_updated.xlsx"
df = pd.read_excel(file_path, skiprows=3)  # Skip headers used for Excel formatting
df = df.iloc[:, :9]  # Keep only relevant columns

# Rename columns for clarity and consistency
df.columns = [
    "Roll No", "Name",
    "Series 1", "Series 2",
    "Assignment 1", "Assignment 2",
    "Series Avg", "Assignment Avg",
    "Internal"
]

# ---------------- Utility Function ----------------
def sanitize_filename(s):
    """
    Convert a string to a safe filename:
    - Remove invalid characters for Windows
    - Trim to 50 characters
    - Handle missing or NaN values
    """
    s = "" if s is None or (isinstance(s, float) and np.isnan(s)) else str(s)
    s = s.strip()
    s = re.sub(r'[\\/*?:"<>|]', "", s)
    return s[:50] if s else "Unknown"

# ---------------- Generate PDF Reports for Each Student ----------------
for _, row in df.iterrows():
    roll = row.get("Roll No", "")
    name = row.get("Name", "")
    roll_str = sanitize_filename(roll)
    name_str = sanitize_filename(name)
    pdf_path = os.path.join(report_dir, f"{roll_str}_{name_str}.pdf")

    # Helper to safely display values in tables
    def val(v):
        return v if (pd.notna(v) and v != "") else "N/A"

    # Extract student marks
    s1 = row.get("Series 1", np.nan)
    s2 = row.get("Series 2", np.nan)
    a1 = row.get("Assignment 1", np.nan)
    a2 = row.get("Assignment 2", np.nan)
    s_avg = row.get("Series Avg", np.nan)
    a_avg = row.get("Assignment Avg", np.nan)
    internal = row.get("Internal", np.nan)

    # Compute fallback averages if missing
    try:
        if pd.isna(s_avg) and not (pd.isna(s1) or pd.isna(s2)):
            s_avg = round((float(s1) + float(s2)) / 2, 2)
    except Exception:
        pass
    try:
        if pd.isna(a_avg) and not (pd.isna(a1) or pd.isna(a2)):
            a_avg = round((float(a1) + float(a2)) / 2, 2)
    except Exception:
        pass
    try:
        if pd.isna(internal) and not (pd.isna(s_avg) or pd.isna(a_avg)):
            internal = round(float(s_avg) + float(a_avg), 2)
    except Exception:
        pass

    # Create PDF report
    with PdfPages(pdf_path) as pdf:
        # Page 1: Tabular summary of student marks
        fig, ax = plt.subplots(figsize=(8.5, 6))
        title_text = f"Progress Report\n{name_str} (Roll No: {roll_str})"
        fig.suptitle(title_text, fontsize=14, fontweight="bold")
        table_data = [
            ["Series 1", val(s1)],
            ["Series 2", val(s2)],
            ["Assignment 1", val(a1)],
            ["Assignment 2", val(a2)],
            ["Series Average", val(s_avg)],
            ["Assignment Average", val(a_avg)],
            ["Internal Total", val(internal)]
        ]
        table = ax.table(cellText=table_data, colLabels=["Metric", "Value"], loc="center", cellLoc="center")
        table.auto_set_font_size(False)
        table.set_fontsize(10)
        table.scale(1.2, 1.2)
        ax.axis("off")  # Hide axes for clean table
        pdf.savefig(fig, bbox_inches="tight")
        plt.close(fig)

        # Page 2: Visual representation of student performance
        fig, axs = plt.subplots(1, 2, figsize=(12, 5))
        fig.suptitle(f"Performance Graphs - {name_str}", fontsize=14, fontweight="bold")

        # Series marks bar chart
        s1_plot = 0 if pd.isna(s1) else float(s1)
        s2_plot = 0 if pd.isna(s2) else float(s2)
        axs[0].bar(["Series 1", "Series 2"], [s1_plot, s2_plot], color=["#ff9999", "#66b3ff"])
        axs[0].set_title("Series Marks")
        axs[0].set_ylim(0, max(40, s1_plot, s2_plot))
        for i, v in enumerate([s1_plot, s2_plot]):
            if v != 0:
                axs[0].text(i, v + 0.5, f"{v:.0f}" if float(v).is_integer() else f"{v:.1f}", ha='center')

        # Assignment marks bar chart
        a1_plot = 0 if pd.isna(a1) else float(a1)
        a2_plot = 0 if pd.isna(a2) else float(a2)
        axs[1].bar(["Assignment 1", "Assignment 2"], [a1_plot, a2_plot], color=["#99ff99", "#ffcc99"])
        axs[1].set_title("Assignment Marks")
        axs[1].set_ylim(0, max(10, a1_plot, a2_plot))
        for i, v in enumerate([a1_plot, a2_plot]):
            if v != 0:
                axs[1].text(i, v + 0.2, f"{v:.0f}" if float(v).is_integer() else f"{v:.1f}", ha='center')

        pdf.savefig(fig, bbox_inches="tight")
        plt.close(fig)

print(f"Saved individual PDF reports to folder: {report_dir}")


Saved individual PDF reports to folder: Student_Reports
